In [ ]:
#  ASR Notes 
# ==========================
#     1/ Store S&P symbols for ALL years in one File , so that easy to Load into dataFramme and slice it
#     2/ gains also store in One FILE for ALL Years , so that we can plot and compare multiple yeras
    
# StackOverFlow good links:
# =============================
# 1/ DF Copy:  http://stackoverflow.com/questions/34682828/pandas-extracting-specific-selected-columns-from-a-dataframe-to-new-dataframe   
  
# 2/ Adding a Row : http://stackoverflow.com/questions/27873190/creating-new-pandas-dataframe-from-existing-dataframe-and-index
        

In [1]:
#import desired packages
import numpy as np
import pandas as pd
from pprint import pprint
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
import urllib.request

def get_year_open_close_price(symbol, year):
    u = "http://chart.finance.yahoo.com/table.csv?s={}&a={}&b={}&c={}&d={}&e={}&f={}&g=d&ignore=.csv"
    month, day = 1, 1  # JAN 1 , first day
    u1 = u.format(symbol, month-1, day, year, month-1, day+5, year)
    
    month, day = 12, 31  # Dec 31 , last day
    u2 = u.format(symbol, month-1, day-5, year, month-1, day, year)
    
    try:
        urllib.request.urlretrieve (u1, "mydat.csv")
        df = pd.read_csv('mydat.csv', header=0)
        price1 = float(df['Open'].tolist()[-1])  # why -1 ? to get first record value by Date
    except: 
        return []  # retrun empty in case of exception

    urllib.request.urlretrieve (u2, "mydat.csv")
    df = pd.read_csv('mydat.csv', header=0)
    price2 = float(df['Close'].tolist()[0])  # why 0 ? to get last record value ny Date
    
    return [ year, symbol, round(price1, 2), round(price2, 2),  round(((price2 - price1) * 100)/price1, 2)]
   


In [3]:
for year in range(2000, 2010):
    rec = get_year_open_close_price('AAPL', year)
    print (rec)

[2000, 'AAPL', 104.87, 14.88, -85.82]
[2001, 'AAPL', 14.88, 21.9, 47.23]
[2002, 'AAPL', 22.05, 14.33, -35.01]
[2003, 'AAPL', 14.36, 21.37, 48.82]
[2004, 'AAPL', 21.55, 64.4, 198.84]
[2005, 'AAPL', 64.78, 71.89, 10.98]
[2006, 'AAPL', 72.38, 84.84, 17.21]
[2007, 'AAPL', 86.29, 198.08, 129.55]
[2008, 'AAPL', 199.27, 85.35, -57.17]
[2009, 'AAPL', 85.88, 210.73, 145.38]


In [34]:
#import SnP Symbol list 
sym_frame = pd.read_csv('/home/asr/Downloads/SnP-companies.csv', header=0)
pprint (sym_frame.head() )

sym_list = sym_frame['Symbol'].tolist()
year = 2014
table = []

for symbol in sym_list:
#for symbol in [ 'MMM', 'IBM', 'ATVI', 'MMM', 'EBAY', 'AAPL', 'MSFT', 'FB']:   #with FB getting error NYSE stock
#for symbol in [ 'MMM', 'IBM', 'ATVI', 'MMM', 'EBAY', 'AAPL', 'MSFT' ]:
    rec = get_year_open_close_price(symbol , year)
    if len(rec) > 0:
        table.append(rec)
    else:
        print(symbol)
   
df = pd.DataFrame(table)    
cols = ['Year', 'Symbol', 'Open', 'Close', 'Gain']
df.columns = cols
df2 = df.sort_values(by='Gain', ascending=False)
df2.to_csv(out_file, index=False)
    

  Symbol                 Name                  Sector
0    MMM           3M Company             Industrials
1    ABT  Abbott Laboratories             Health Care
2   ABBV               AbbVie             Health Care
3    ACN        Accenture plc  Information Technology
4   ATVI  Activision Blizzard  Information Technology
AA
CFG
CSRA
FTV
HPE
KHC
NAVI
NEE
PYPL
QRVO
SYF
UA
WRK
WLTW


/home/asr/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:21: FutureWarning: by argument to sort_index is deprecated, pls use .sort_values(by=...)


In [36]:
df2 = df.sort_values(by='Gain', ascending=False) 
df3 = df2.query('Gain > 80 or Gain < -40')
df3

,Year,Symbol,Open,Close,Gain
393,2014,SWKS,28.30,72.71,156.93
398,2014,LUV,18.85,42.32,124.51
28,2014,AAL,25.07,53.63,113.92
156,2014,EA,22.90,47.02,105.33
155,2014,EW,65.68,127.38,93.94
55,2014,AVGO,52.85,100.59,90.33
277,2014,MNK,52.26,99.03,89.49
131,2014,DAL,27.26,49.19,80.45
161,2014,EOG,166.57,92.07,-44.73
110,2014,CTSH,100.25,52.66,-47.47


In [15]:
def get_SnP_symbols_for_year( year):
    
    try:
        df = pd.read_csv('sp-add-remove-final.csv', header=0)
        df2 = df.query(' Year == @year')
        return df2
    except: 
        return []  # retrun empty in case of exception
    
df3 = get_SnP_symbols_for_year (2016)
df3

,Date,Added,Removed,Year
1,12/01/2016,MAA,OI,2016
2,12/01/2016,EVHC,LM,2016
3,09/30/2016,COTY,DO,2016
4,09/22/2016,COO,HOT,2016
5,09/08/2016,CHTR,EMC,2016
6,09/06/2016,MTD,JCI,2016
7,07/05/2016,FTV,CPGX,2016
8,07/01/2016,LNT,GAS,2016
9,07/01/2016,ALB,TE,2016
10,06/22/2016,FBHS,CVC,2016


In [32]:
import traceback

def create_SnP_symbols_by_year():
    
    try:
        df = pd.read_csv('SnP-10-years.csv', header=0)
        df.to_csv('SnP-10-years-dfready.csv', index=False)  # use this CREATEd DF Ready file for later use
        return True
    except: 
        traceback.print_exc()
        return False  # retrun empty in case of exception
    
ret = create_SnP_symbols_by_year()
print(ret)

True
